In [1]:
import sagemaker
from sagemaker.tensorflow import TensorFlow
import os
import boto3
import botocore
import re
from sagemaker import get_execution_role
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

tf.enable_eager_execution()
tf.set_random_seed(0)

# initialise S3 information

role = get_execution_role()
region = boto3.Session().region_name

bucket='sagemaker-cert-bucket' # put your s3 bucket name here, and create s3 bucket
prefix = 'sagemaker/tensorflow-regression-abalone'

# customize to your bucket where you have stored the data
bucket_path = 'https://s3-{}.amazonaws.com/{}'.format(region,bucket)

# download the dataset from S3

BUCKET_NAME = 'sagemaker-cert-bucket' # replace with your bucket name
KEY = 'abalone/abalone.csv' # replace with your object key

s3 = boto3.resource('s3')

try:
    s3.Bucket(BUCKET_NAME).download_file(KEY, 'abalone.csv')
except botocore.exceptions.ClientError as e:
    if e.response['Error']['Code'] == "404":
        print("The object does not exist.")
    else:
        raise
        
# prepare the data

data = pd.read_csv("abalone.csv", header=0)
column_names = ['SEX', 'LEN', 'DIA', 'HEI', 'W1', 'W2', 'W3', 'W4', 'RIN']

data.columns = column_names

data = pd.get_dummies(data)

labels = data["RIN"]
data = data.drop(['RIN'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.33, random_state=42)

mean = X_train.mean(axis=0)
std = X_train.std(axis=0)
X_train = (X_train - mean) / std
X_test = (X_test - mean) / std

# upload data

X_train.to_csv('X_train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/X_train.csv')).upload_file('X_train.csv')
y_train.to_csv('y_train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/y_train.csv')).upload_file('y_train.csv')
s3_input_train = sagemaker.s3_input(s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='csv')

X_test.to_csv('X_test.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'test/X_test.csv')).upload_file('X_test.csv')
y_test.to_csv('y_test.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'test/y_test.csv')).upload_file('y_test.csv')
s3_input_test = sagemaker.s3_input(s3_data='s3://{}/{}/test'.format(bucket, prefix), content_type='csv')


inputs = {'train': s3_input_train,
          'test': s3_input_test}

In [2]:
train_instance_type = 'ml.c4.xlarge'
model_dir = '/opt/ml/model'
hyperparameters = {'epochs': 500, 'batch_size': 32}

estimator = TensorFlow(entry_point='script.py',
                       model_dir=model_dir,
                       train_instance_type=train_instance_type,
                       train_instance_count=1,
                       hyperparameters=hyperparameters,
                       role=sagemaker.get_execution_role(),
                       base_job_name='tf-eager-scriptmode-abalone',
                       framework_version='1.12.0',
                       py_version='py3')

estimator.fit(inputs)
!aws s3 cp {estimator.model_data} ./model/model.tar.gz
!tar -xvzf ./model/model.tar.gz -C ./model

INFO:sagemaker:Creating training-job with name: tf-eager-scriptmode-abalone-2019-04-17-19-51-47-929


2019-04-17 19:51:48 Starting - Starting the training job...
2019-04-17 19:51:53 Starting - Launching requested ML instances......
2019-04-17 19:52:59 Starting - Preparing the instances for training.........
2019-04-17 19:54:33 Downloading - Downloading input data
2019-04-17 19:54:33 Training - Training image download completed. Training in progress.
2019-04-17 19:54:36,637 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2019-04-17 19:54:36,642 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2019-04-17 19:54:36,955 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2019-04-17 19:54:36,973 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2019-04-17 19:54:36,985 sagemaker-containers INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "test": "/opt/ml/input/data/test",
        "train": "/opt/ml

#015  32/2797 [..............................] - ETA: 0s - loss: nan - acc: 0.0000e+00#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#0151120/2797 [===========>..................] - ETA: 0s - loss: nan - acc: 0.0000e+00#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#0152208/2797 [======================>.......] - ETA: 0s - loss: nan - acc: 0.0000e+00#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#0

#015  32/2797 [..............................] - ETA: 0s - loss: nan - acc: 0.0000e+00#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#0151120/2797 [===========>..................] - ETA: 0s - loss: nan - acc: 0.0000e+00#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#0152080/2797 [=====================>........] - ETA: 0s - loss: nan - acc: 0.0000e+00#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#0

#015  32/2797 [..............................] - ETA: 0s - loss: nan - acc: 0.0000e+00#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 576/2797 [=====>........................] - ETA: 0s - loss: nan - acc: 0.0000e+00#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#0151152/2797 [===========>..................] - ETA: 0s - loss: nan - acc: 0.0000e+00#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#0

...

#015  32/2797 [..............................] - ETA: 0s - loss: nan - acc: 0.0000e+00#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#0151184/2797 [===========>..................] - ETA: 0s - loss: nan - acc: 0.0000e+00#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#0152272/2797 [=======================>......] - ETA: 0s - loss: nan - acc: 0.0000e+00#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#0

#015  32/2797 [..............................] - ETA: 0s - loss: nan - acc: 0.0000e+00#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#0151120/2797 [===========>..................] - ETA: 0s - loss: nan - acc: 0.0000e+00#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#0152208/2797 [======================>.......] - ETA: 0s - loss: nan - acc: 0.0000e+00#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#0

#015  32/2797 [..............................] - ETA: 0s - loss: nan - acc: 0.0000e+00#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#0151152/2797 [===========>..................] - ETA: 0s - loss: nan - acc: 0.0000e+00#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#0152240/2797 [=======================>......] - ETA: 0s - loss: nan - acc: 0.0000e+00#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#0

#015  32/2797 [..............................] - ETA: 0s - loss: nan - acc: 0.0000e+00#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#0151184/2797 [===========>..................] - ETA: 0s - loss: nan - acc: 0.0000e+00#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#0152272/2797 [=======================>......] - ETA: 0s - loss: nan - acc: 0.0000e+00#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#0


2019-04-17 19:56:16 Uploading - Uploading generated training model
2019-04-17 19:56:16 Completed - Training job completed
2019-04-17 19:56:08,474 sagemaker-containers INFO     Reporting training SUCCESS
Billable seconds: 116
download: s3://sagemaker-eu-central-1-769521381365/tf-eager-scriptmode-abalone-2019-04-17-19-51-47-929/output/model.tar.gz to model/model.tar.gz
1555530968/
1555530968/assets/
1555530968/assets/saved_model.json
1555530968/variables/
1555530968/variables/checkpoint
1555530968/variables/variables.index
1555530968/variables/variables.data-00000-of-00001
1555530968/saved_model.pb


In [ ]:
predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m5.xlarge')

INFO:sagemaker:Creating model with name: tf-eager-scriptmode-abalone-2019-04-17-19-51-47-929
INFO:sagemaker:Creating endpoint with name tf-eager-scriptmode-abalone-2019-04-17-19-51-47-929


---------

In [ ]:
sagemaker.Session().delete_endpoint(predictor.endpoint)